<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## Data Processing Workflow

In this workflow within PyCCAPT, we can crop the data, do the voltage and bowl calibration, calculate the 3d reconstruction, and do the ranging.
</div>

In [1]:
# Activate intractive functionality of matplotlib
%matplotlib ipympl
# Activate auto reload 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# import libraries
import os
import numpy as np
import pandas as pd
import subprocess
from ipywidgets import widgets
from IPython.display import display
from ipywidgets import fixed, interact_manual
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

# Local module and scripts
from pyccapt.calibration.calibration import widgets as wd
from pyccapt.calibration.data_tools import data_tools, data_loadcrop
from pyccapt.calibration.tutorials.tutorials_helpers import helper_calibration
from pyccapt.calibration.tutorials.tutorials_helpers import helper_data_loader
from pyccapt.calibration.tutorials.tutorials_helpers import helper_temporal_crop
from pyccapt.calibration.tutorials.tutorials_helpers import helper_special_crop
from pyccapt.calibration.tutorials.tutorials_helpers import helper_t_0_tune
from pyccapt.calibration.tutorials.tutorials_helpers import helper_mc_plot
from pyccapt.calibration.tutorials.tutorials_helpers import helper_3d_reconstruction
from pyccapt.calibration.tutorials.tutorials_helpers import helper_ion_selection
from pyccapt.calibration.tutorials.tutorials_helpers import helper_visualization
from pyccapt.calibration.tutorials.tutorials_helpers import helper_ion_list
from pyccapt.calibration.calibration import ion_selection
from pyccapt.calibration.calibration import share_variables
from pyccapt.calibration.leap_tools.leap_tools import read_rrng

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
In case of recieving the error about pytable library, you have to install the pytables library with conda command. to do that you can open a new cell and copy the line below in it. Then just run it like other cells. The pytables library will be innstalled.
    
`!conda install --yes --prefix {sys.prefix} pytables`
</div>


<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
In the cell below we create variable object. This object is used in many of the functions to share the data between functions in as easy way. 
</div>


In [2]:
variables = share_variables.Variables()

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
    
By clicking on the button below, you can select the dataset file you want to work with. The dataset file can be in various formats, including HDF5, EPOS, POS, ATO, and CSV. The cropped data will be saved in the same directory as the original dataset file in a new directory nammed load_crop. The name of the cropped dataset file will be the same as the original dataset file. The figures will be saved in the same directory as the dataset file.
</div>

In [3]:
button = widgets.Button(description='load dataset')

@button.on_click
def open_file_on_click_r(b):
    global dataset_path
    folder_path = variables.last_directory
    script = '..//..//data_tools//run_dataset_path_qt.py'
    cmd = f"python {script} {folder_path}"
    result = subprocess.run(cmd, capture_output=True, text=True, shell=True)
    dataset_path = result.stdout.strip()
    variables.last_directory = dataset_path

button

Button(description='load dataset', style=ButtonStyle())

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## ROI Selection and Data Cropping
    

</div>



<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

    
From the dropdown lists below, you can select the instrument specifications of the dataset. The instrument specifications are the same as the ones used for the calibration process. Data mode is specify the dataset structure. The dataset can be in raw or calibrated mode. The flight path length is the distance between the sample and the detector. The t0 is the time of flight of the ions with the lowest mass-to-charge ratio. The maximum mass-to-charge ratio is the maximum mass-to-charge ratio of tat you want to plot. You can also change it in te related cells. The detector diameter is the diameter of the detector.
</div>


In [4]:
# create an object for selection of instrument specifications of the dataset
tdc, pulse_mode, flight_path_length, t0, max_mc, det_diam = wd.dataset_instrument_specification_selection()

# Display lists and comboboxes to selected instrument specifications
display(tdc, pulse_mode, flight_path_length, t0, det_diam, max_mc)

Dropdown(description='Data mode:', options=('pyccapt', 'leap_epos', 'leap_pos', 'leap_apt', 'ato_v6'), value='…

Dropdown(description='Pulse mode:', options=('voltage', 'laser'), value='voltage')

FloatText(value=110.0, description='Flight path length:')

FloatText(value=38.0, description='t0:')

FloatText(value=80.0, description='Detector diameter:')

FloatText(value=400.0, description='Max mc:')

In [14]:
z = - variables.data['z (nm)'].to_numpy()
dz = np.diff(z)
np.min(dz)

-8.337142944335938

In [13]:
helper_data_loader.load_data(dataset_path, max_mc.value, flight_path_length.value, pulse_mode.value, tdc.value, variables)
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)
display(variables.data)
display(variables.range_data)

The maximum possible TOF is: 4099 ns
The data will be saved on the path: T:/Monajem/manuscript/PyCCAPT_A_Python_Package_for_Open-Source_Atom_Probe_Instrument_Control_and_Data_Calibration/Data/W_LEAO_5000_XS/R5076_80365/data_processing/
The dataset name after saving is: R5076_80365
The figures will be saved on the path: T:/Monajem/manuscript/PyCCAPT_A_Python_Package_for_Open-Source_Atom_Probe_Instrument_Control_and_Data_Calibration/Data/W_LEAO_5000_XS/R5076_80365/data_processing/
Total number of Ions: 13798893
The maximum possible time of flight is: 4099


,x (nm),y (nm),z (nm),mc (Da),mc_uc (Da),high_voltage (V),pulse,t (ns),t_c (ns),x_det (cm),y_det (cm),delta_p,multi,start_counter
0,3.839731,-5.632716,-1.057883,0.0,0.0,4760.439941,0.0,959.159912,0.0,1.001764,-1.626275,0,1,0
1,-3.405475,7.871015,-1.699164,0.0,0.0,4760.439941,0.0,969.302979,0.0,-1.050907,2.200255,443,1,0
2,-10.753205,-0.979574,-2.761261,0.0,0.0,4760.439941,0.0,984.524902,0.0,-3.137601,-0.313776,469,1,0
3,-4.936064,10.501042,-3.223996,0.0,0.0,4760.439941,0.0,988.666992,0.0,-1.481855,2.946428,1208,1,0
4,8.569964,-4.414245,-2.170618,0.0,0.0,4760.439941,0.0,981.073975,0.0,2.370212,-1.304847,184,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13798888,-11.058736,4.679689,-189.755890,0.0,0.0,9942.240234,0.0,100.990997,0.0,-1.561240,0.589286,219,1,0
13798889,-14.238710,-1.687609,-190.436890,0.0,0.0,9942.240234,0.0,661.300049,0.0,-2.003528,-0.267857,238,1,0
13798890,6.756890,-16.737640,-191.801331,0.0,0.0,9942.240234,0.0,1324.916016,0.0,0.842994,-2.322704,335,1,0
13798891,13.953911,-16.103716,-193.305801,0.0,0.0,9942.240234,0.0,557.759033,0.0,1.829637,-2.246173,314,1,0


,name,ion,mass,mc,mc_low,mc_up,color,element,complex,isotope,charge
0,H1,${}^{1}H^{+}$,1.0100,0.958999,0.926484,1.318946,#b2aa2d,[H],[1],[1],1.0
1,H2,${}^{1}H_{2}^{+}$,2.0200,2.057186,1.938820,2.343547,#b2aa2d,[H],[2],[1],1.0
2,C1,${}^{13}C^{+}$,13.0000,13.438397,13.405882,13.602113,#daa460,[C],[1],[13],1.0
3,Ga1,${}^{69}Ga^{2+}$,34.4650,34.403785,34.248626,34.800526,#58c7ac,[Ga],[1],[69],2.0
4,W1,${}^{180}W^{4+}$,44.9875,45.385656,45.328611,45.598429,#ab70d5,[W],[1],[180],4.0
5,W1,${}^{182}W^{4+}$,45.4875,45.685161,45.640381,45.848877,#ab70d5,[W],[1],[182],4.0
6,W1,${}^{183}W^{4+}$,45.7375,45.884832,45.852316,46.146663,#ab70d5,[W],[1],[183],4.0
7,W1,${}^{184}W^{4+}$,45.9875,46.384007,46.314699,46.645839,#ab70d5,[W],[1],[184],4.0
8,W1,${}^{180}W^{3+}$,59.9833,59.961592,59.806432,60.198895,#ab70d5,[W],[1],[180],3.0
9,W1,${}^{182}W^{3+}$,60.6500,60.560603,60.429973,60.822435,#ab70d5,[W],[1],[182],3.0


<div style="margin: 0 auto; padding: 20px;">

Load a pre saved range file if you want from cell below
</div>


In [6]:
button_r = widgets.Button(description='load range dataset')

@button_r.on_click
def open_file_on_click_r(b):
    global range_path
    folder_path = variables.last_directory
    script = '..//..//data_tools//run_dataset_path_qt.py'
    cmd = f"python {script} {folder_path}"
    result = subprocess.run(cmd, capture_output=True, text=True, shell=True)
    range_path = result.stdout.strip()
    variables.last_directory = range_path

button_r

Button(description='load range dataset', style=ButtonStyle())

In [7]:
if 'range_path' in globals():
    _, file_extension = os.path.splitext(range_path)
    if file_extension == '.rrng':
        variables.range_data = read_rrng(range_path)
    else:
        variables.range_data = pd.read_hdf(range_path, mode='r')
display(variables.range_data)

,name,ion,mass,mc,mc_low,mc_up,color,element,complex,isotope,charge
0,unranged0,un,0.0,0.0,0.0,400.0,#000000,[unranged],[0],[0],0


<div style="margin: 0 auto; padding: 20px;">

# Temporal crop

Select the data by drawing a rectangle over the experiment history. Experiment history is a 2D histogram of the time of flight of the ions versus sequence of evaporation. The experiment history is plotted by clicking on the button below te cell.
</div>


In [8]:
helper_temporal_crop.call_plot_crop_experiment(variables, pulse_mode.value)

Output()

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

# Spatial crop

Select the region of maximum concentration of Ions in the below plotted graph to utilize relevant data. To crop you can draw a circle over the filed desorption map. The field desorption map is a 2D histogram of the time of flight of the ions versus the position of the ions on the detector. The field desorption map is plotted by clicking on the button below the cell.
</div>

In [9]:
helper_special_crop.call_plot_crop_fdm(variables)

Output()

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Calculate pulses since the last event pulse and ions per pulse. The percentage of loss in ROI selection process will be printed.
</div>

In [10]:
# maximum value of the tdc counter
if tdc.value == 'pyccapt':
    if not (variables.data['delta_p'] != 0).any() or not (variables.data['multi'] != 0).any():
        if (variables.data['start_counter'] != 0).any():
            max_start_counter = max(variables.data['start_counter'])
            # if (variables.data['pulse_pi'] == 0).all() or (variables.data['ion_pp'] == 0).all():
            pulse_pi, ion_pp = data_loadcrop.calculate_ppi_and_ipp(variables.data, max_start_counter)
            # add two calculated array to the croped dataset
            variables.data['delta_p'] = pulse_pi.astype(np.uintc)
            variables.data['multi'] = ion_pp.astype(np.uintc)
    
    # save the cropped data
    print('tof Crop Loss {:.2f} %'.format((100 - (len(variables.data) / len(variables.data_backup)) * 100)))
#percentage of double event per pulse
ion_pp = variables.data['multi'].to_numpy()
print('percentage of multihit event per pulse', len(ion_pp[ion_pp != 1]) / float(len(ion_pp)) * 100)

Progress: 0% complete
Progress: 20% complete
Progress: 40% complete
Progress: 60% complete
Progress: 80% complete
tof Crop Loss 49.83 %
percentage of multihit event per pulse 1.5902402482743863


<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

In the next cell by changing the t<sub>0</sub> value you can correct the position of H<sub>1</sub> or any other known peak. this correction would be helpful for the position of the peaks in the m/c calibration process.
</div>

In [11]:
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)
variables.data

The maximum possible time of flight is: 8062


,x (nm),y (nm),z (nm),mc (Da),mc_uc (Da),high_voltage (V),pulse,t (ns),t_c (ns),x_det (cm),y_det (cm),delta_p,multi,start_counter
0,0.0,0.0,0.0,0.0,0.0,5915.231445,150.0,5175.8193,0.0,-2.101450,-3.169670,0,1,224651
1,0.0,0.0,0.0,0.0,0.0,5915.231445,150.0,199.5550,0.0,-1.320550,0.964226,1,1,224652
2,0.0,0.0,0.0,0.0,0.0,5915.231445,150.0,1673.7403,0.0,1.566120,-1.120790,1,1,224653
3,0.0,0.0,0.0,0.0,0.0,5915.231445,150.0,762.1776,0.0,-1.738850,1.584940,1,1,224654
4,0.0,0.0,0.0,0.0,0.0,5915.231445,150.0,2446.0465,0.0,-0.775419,2.660880,1,1,224655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186885,0.0,0.0,0.0,0.0,0.0,4449.818848,183.0,2034.1188,0.0,1.956030,-2.828240,1,1,455515
186886,0.0,0.0,0.0,0.0,0.0,4449.818848,183.0,2111.8709,0.0,-1.395810,-2.664310,1,1,455516
186887,0.0,0.0,0.0,0.0,0.0,4449.818848,183.0,2098.5070,0.0,1.585810,-2.296330,2,1,455518
186888,0.0,0.0,0.0,0.0,0.0,4449.818848,183.0,2001.6230,0.0,0.263876,2.806310,1,1,455519


In [12]:
helper_t_0_tune.call_fine_tune_t_0(variables, flight_path_length, pulse_mode, t0)

Output()

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Remove the data with m/c greater than max m/c and x, y, t = 0. Also add the needed colums for calibration. The data types of the final cropped dataset is displayed below.

</div>

In [13]:
# add the columns in the dataset for x, y, z, mc, mc calibrated, and t calibrated
helper_data_loader.add_columns(variables, max_mc)
# save data temporarily
data_tools.save_data(variables.data, variables, hdf=True, epos=False, pos=False, csv=False, temp=True)
display(variables.data)
display(variables.data.dtypes)

The number of data over max_mc: 0
The number of data with having t, x, and y equal to zero is: 0


,x (nm),y (nm),z (nm),mc (Da),mc_uc (Da),high_voltage (V),pulse,t (ns),t_c (ns),x_det (cm),y_det (cm),delta_p,multi,start_counter
0,0.0,0.0,0.0,0.0,951.257176,5915.231445,150.0,5175.8193,0.0,-2.101450,-3.169670,0,1,224651
1,0.0,0.0,0.0,0.0,1.057918,5915.231445,150.0,199.5550,0.0,-1.320550,0.964226,1,1,224652
2,0.0,0.0,0.0,0.0,100.264758,5915.231445,150.0,1673.7403,0.0,1.566120,-1.120790,1,1,224653
3,0.0,0.0,0.0,0.0,19.746714,5915.231445,150.0,762.1776,0.0,-1.738850,1.584940,1,1,224654
4,0.0,0.0,0.0,0.0,214.027510,5915.231445,150.0,2446.0465,0.0,-0.775419,2.660880,1,1,224655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186885,0.0,0.0,0.0,0.0,110.537356,4449.818848,183.0,2034.1188,0.0,1.956030,-2.828240,1,1,455515
186886,0.0,0.0,0.0,0.0,120.313571,4449.818848,183.0,2111.8709,0.0,-1.395810,-2.664310,1,1,455516
186887,0.0,0.0,0.0,0.0,119.238784,4449.818848,183.0,2098.5070,0.0,1.585810,-2.296330,2,1,455518
186888,0.0,0.0,0.0,0.0,108.272003,4449.818848,183.0,2001.6230,0.0,0.263876,2.806310,1,1,455519


x (nm)              float64
y (nm)              float64
z (nm)              float64
mc (Da)             float64
mc_uc (Da)          float64
high_voltage (V)    float64
pulse               float64
t (ns)              float64
t_c (ns)            float64
x_det (cm)          float64
y_det (cm)          float64
delta_p              uint32
multi                uint32
start_counter        uint32
dtype: object

-------------------------------------------------------
-------------------------------------------------------

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## Time-of-Flight/Mass-to-charge Calibration
</div>

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
    
Below you can plot the ToF histogram of the dataset. You can select the peak range of the data you want to plot by drawing a rectangle over peak with holding left click. After that you should apply the voltage and bowl calibration. These three steps should be repeated until you see no improvement in the peak resolution. You can also save the calibration by clicking on the save button. The saved calibration will be used for the next steps.

</div>

In [14]:
# exctract needed data from Pandas data frame as an numpy array
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)

The maximum possible time of flight is: 8062


In [15]:
helper_calibration.call_voltage_bowl_calibration(variables, det_diam.value, flight_path_length.value, pulse_mode.value)

cannot find the maximum for i, j: 22 31
cannot find the maximum for i, j: 27 26
cannot find the maximum for i, j: 32 21
cannot find the maximum for i, j: 22 31
cannot find the maximum for i, j: 27 26
cannot find the maximum for i, j: -38 16
cannot find the maximum for i, j: -28 -34
cannot find the maximum for i, j: -28 -33
cannot find the maximum for i, j: 27 27
cannot find the maximum for i, j: -32 -27
cannot find the maximum for i, j: -29 -30
cannot find the maximum for i, j: 1 15
cannot find the maximum for i, j: 1 12
cannot find the maximum for i, j: 7 12
cannot find the maximum for i, j: 34 -21
cannot find the maximum for i, j: -38 12
cannot find the maximum for i, j: -32 -27
cannot find the maximum for i, j: -5 15
cannot find the maximum for i, j: -2 15
cannot find the maximum for i, j: 1 12
cannot find the maximum for i, j: 1 18
cannot find the maximum for i, j: 28 -30
cannot find the maximum for i, j: 37 9


In [16]:
variables.dld_t_calib_backup = np.copy(variables.dld_t_calib)
variables.mc_calib_backup = np.copy(variables.mc_calib)

In [17]:
helper_ion_list.call_ion_list(variables, selector='peak')

In [18]:
variables.data['mc (Da)'] = variables.mc_calib
variables.data['t_c (ns)'] = variables.dld_t_calib
# Remove negative mc
threshold = 0
mc_t = variables.data['mc (Da)'].to_numpy()
mc_t_mask = (mc_t <= threshold)
print('The number of ions with negative mc are:', len(mc_t_mask[mc_t_mask==True]))
variables.data.drop(np.where(mc_t_mask)[0], inplace=True)
variables.data.reset_index(inplace=True, drop=True)
# save data temporarily
data_tools.save_data(variables.data, variables, hdf=True, epos=False, pos=False, csv=False, temp=True)

The number of ions with negative mc are: 0


In [19]:
helper_mc_plot.call_mc_plot(variables, selector='None')

Output()

-------------------------------------------------------
-------------------------------------------------------

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## 3D Reconstruction
After bowl and voltage calibration we are ready to calculate the 3d reconstruction. In this workflow we calculate the reconstructed x,y,z and then plot the 3d, heatmap, projection plots and mass-to-charge histogram.

</div>

In [20]:
# exctract needed data from Pandas data frame as a numpy array
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)

display(variables.data)

The maximum possible time of flight is: 12435


,x (nm),y (nm),z (nm),mc (Da),mc_uc (Da),high_voltage (V),pulse,t (ns),t_c (ns),x_det (cm),y_det (cm),delta_p,multi,start_counter
0,0.0,0.0,0.0,1.004139,0.786007,7767.467773,38548.199219,231.0663,233.957082,-0.756595,1.213980,0.0,0.0,0
1,0.0,0.0,0.0,89.755198,76.328761,7767.467773,38548.199219,1999.5467,1999.487180,0.406925,-3.632460,0.0,0.0,0
2,0.0,0.0,0.0,130.713750,110.987143,7767.467773,38548.199219,2398.2223,2407.584075,-3.072740,-0.000798,0.0,0.0,0
3,0.0,0.0,0.0,567.301917,479.321347,7767.467773,38548.199219,4928.1093,4987.451239,-0.627502,1.692260,0.0,0.0,0
4,0.0,0.0,0.0,2.532462,2.044473,7767.467773,38548.199219,353.3698,357.056217,-0.308264,2.512430,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507188,0.0,0.0,0.0,92.861373,78.118635,6539.762207,47708.601562,2192.0636,2033.345777,0.954383,2.465790,0.0,0.0,0
507189,0.0,0.0,0.0,90.845656,77.369983,6539.762207,47708.601562,2183.5901,2011.439402,-2.384750,-1.622630,0.0,0.0,0
507190,0.0,0.0,0.0,92.927556,78.881532,6539.762207,47708.601562,2206.9331,2034.060981,1.521890,-2.774210,0.0,0.0,0
507191,0.0,0.0,0.0,182.613097,154.149304,6539.762207,47708.601562,3053.0185,2840.951991,0.674327,0.395968,0.0,0.0,0


<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

You have to select the main element in your sample from the from dropdown below.
</div>

In [12]:
element_selected = wd.density_field_selection()
display(element_selected)

Dropdown(description='Element', options=((1, 'H', 52.7, 0.0), (2, 'He', 28.7, 0.0), (3, 'Li', 46.3, 14.0), (4,…

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

In case that yopu face error about plotly library, like javascripts error. You have check your jupyter lab version is compatibale with the plotly extenstion. Sometimes running `jupyter lab build` command fix the proble.
</div>

In [32]:
helper_3d_reconstruction.call_x_y_z_calculation(variables, flight_path_length, element_selected=element_selected)

Output()

In [ ]:
variables.x

In [23]:
variables.data['x (nm)'] = variables.x
variables.data['y (nm)'] = variables.y
variables.data['z (nm)'] = variables.z
# save data temporarily
data_tools.save_data(variables.data, variables, hdf=True, epos=False, pos=False, csv=False, temp=True)

-------------------------------------------------------
-------------------------------------------------------

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## Ion Selection and Rangging

Users can choose ions using peak and element finders, manually add ions, customize ion colors, and create histograms. Histograms can be generated for selected ranges and areas, and figures can be saved. Additionally, users have the option to save both statistics and  range data.

</div>

In [6]:
# exctract needed data from Pandas data frame as an numpy array
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)

display(variables.data)

The maximum possible time of flight is: 4099


,x (nm),y (nm),z (nm),mc (Da),mc_uc (Da),high_voltage (V),pulse,t (ns),t_c (ns),x_det (cm),y_det (cm),delta_p,multi,start_counter
0,3.839731,-5.632716,-1.057883,91.974541,0.0,4760.439941,0.0,959.159912,0.0,1.001764,-1.626275,0,1,0
1,-3.405475,7.871015,-1.699164,91.988487,0.0,4760.439941,0.0,969.302979,0.0,-1.050907,2.200255,443,1,0
2,-10.753205,-0.979574,-2.761261,90.956039,0.0,4760.439941,0.0,984.524902,0.0,-3.137601,-0.313776,469,1,0
3,-4.936064,10.501042,-3.223996,91.374977,0.0,4760.439941,0.0,988.666992,0.0,-1.481855,2.946429,1208,1,0
4,8.569964,-4.414245,-2.170618,92.762627,0.0,4760.439941,0.0,981.073975,0.0,2.370212,-1.304847,184,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13798888,-11.058736,4.679689,-189.755890,2.160229,0.0,9942.240234,0.0,100.990997,0.0,-1.561240,0.589286,219,1,0
13798889,-14.238710,-1.687609,-190.436890,90.923241,0.0,9942.240234,0.0,661.300049,0.0,-2.003528,-0.267857,238,1,0
13798890,6.756890,-16.737640,-191.801331,357.720978,0.0,9942.240234,0.0,1324.916016,0.0,0.842994,-2.322704,335,1,0
13798891,13.953911,-16.103716,-193.305801,62.000912,0.0,9942.240234,0.0,557.759033,0.0,1.829637,-2.246173,314,1,0


In [7]:
display(variables.data.dtypes)

x (nm)              float32
y (nm)              float32
z (nm)              float32
mc (Da)             float32
mc_uc (Da)          float64
high_voltage (V)    float32
pulse               float32
t (ns)              float32
t_c (ns)            float64
x_det (cm)          float32
y_det (cm)          float32
delta_p               int32
multi                 int32
start_counter         int32
dtype: object

In [8]:
helper_ion_selection.call_ion_selection(variables)

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Now we can also show the color in the range dataframe

</div>

In [9]:
display(variables.range_data.style.applymap(ion_selection.display_color, subset=['color']))

,name,ion,mass,mc,mc_low,mc_up,color,element,complex,isotope,charge
0,H1,${}^{1}H^{+}$,1.010000,0.958999,0.926484,1.318946,#b2aa2d,['H'],[1],[1],1.000000
1,H2,${}^{1}H_{2}^{+}$,2.020000,2.057186,1.938820,2.343547,#b2aa2d,['H'],[2],[1],1.000000
2,C1,${}^{13}C^{+}$,13.000000,13.438397,13.405882,13.602113,#daa460,['C'],[1],[13],1.000000
3,Ga1,${}^{69}Ga^{2+}$,34.465000,34.403785,34.248626,34.800526,#58c7ac,['Ga'],[1],[69],2.000000
4,W1,${}^{180}W^{4+}$,44.987500,45.385656,45.328611,45.598429,#ab70d5,['W'],[1],[180],4.000000
5,W1,${}^{182}W^{4+}$,45.487500,45.685161,45.640381,45.848877,#ab70d5,['W'],[1],[182],4.000000
6,W1,${}^{183}W^{4+}$,45.737500,45.884832,45.852316,46.146663,#ab70d5,['W'],[1],[183],4.000000
7,W1,${}^{184}W^{4+}$,45.987500,46.384007,46.314699,46.645839,#ab70d5,['W'],[1],[184],4.000000
8,W1,${}^{180}W^{3+}$,59.983300,59.961592,59.806432,60.198895,#ab70d5,['W'],[1],[180],3.000000
9,W1,${}^{182}W^{3+}$,60.650000,60.560603,60.429973,60.822435,#ab70d5,['W'],[1],[182],3.000000


In [10]:
variables.range_data.dtypes

name        object
ion         object
mass       float64
mc         float64
mc_low     float64
mc_up      float64
color       object
element     object
complex     object
isotope     object
charge     float64
dtype: object

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Save the range in the hdf5 and csv format.

</div>

In [11]:
interact_manual_range = interact_manual.options(manual_name="save range")
interact_manual_range(data_tools.save_range, variables=fixed(variables));

interactive(children=(Button(description='save range', style=ButtonStyle()), Output()), _dom_classes=('widget-…

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Save the cropped dataset. You can specify te output format from list below. The output formats are HDF5, EPOS, POS, ATO, and CSV. The output file will be saved in the same directory as the original dataset file in a new directory nammed load_crop.
</div>

In [26]:
interact_manual_data = interact_manual.options(manual_name="save data")
interact_manual_data(data_tools.save_data, data=fixed(variables.data), variables=fixed(variables),
                name=widgets.Text(value=variables.result_data_name),
                hdf=widgets.Dropdown(options=[('True', True), ('False', False)]),
                epos=widgets.Dropdown(options=[('False', False), ('True', True)]),
                pos=widgets.Dropdown(options=[('False', False), ('True', True)]),
                ato_6v=widgets.Dropdown(options=[('False', False), ('True', True)]),
                csv=widgets.Dropdown(options=[('False', False), ('True', True)]),
               temp=fixed(False));

interactive(children=(Text(value='data_226_Feb-18-2025_08-38_W_Poly', continuous_update=False, description='na…

-------------------------------------------------------
-------------------------------------------------------

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
    
# Visualization
    
</div>

In [28]:
variables.data

,x (nm),y (nm),z (nm),mc (Da),mc_uc (Da),high_voltage (V),pulse,t (ns),t_c (ns),x_det (cm),y_det (cm),delta_p,multi,start_counter
0,0.0,0.0,0.0,41.025871,41.025871,5742.041504,135.0,1109.4928,1250.892924,0.601300,0.336662,0,1,277721
1,0.0,0.0,0.0,1171.091728,1171.091728,5742.041504,135.0,5834.8182,6543.173519,2.223430,0.367079,1,1,277722
2,0.0,0.0,0.0,502.944900,502.944900,5742.041504,135.0,3863.2142,4300.214193,-1.105310,2.946750,1,1,277723
3,0.0,0.0,0.0,363.544050,363.544050,5742.041504,135.0,3309.9606,3630.389223,-3.405260,-1.566110,2,1,277725
4,0.0,0.0,0.0,309.143882,309.143882,5742.041504,135.0,3043.0587,3361.723500,-3.323370,0.698086,1,1,277726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399388,0.0,0.0,0.0,104.011627,104.011627,3948.260498,175.0,2118.0675,1971.540548,-0.225676,2.317360,1,1,769665
399389,0.0,0.0,0.0,113.201519,113.201519,3948.260498,175.0,2235.6945,2044.608032,-3.626180,0.582122,1,1,769666
399390,0.0,0.0,0.0,111.691448,111.691448,3948.260498,175.0,2176.5069,2038.364964,0.357275,0.483394,2,1,769668
399391,0.0,0.0,0.0,105.538162,105.538162,3948.260498,175.0,2131.1059,1980.932617,-1.586550,1.493650,1,1,769669


# helper_visualization.call_visualization(variables)

In [32]:
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)

The maximum possible time of flight is: 8062


In [31]:
helper_visualization.call_visualization(variables)

In [ ]:
variables.plotly_3d_reconstruction.show()

In [ ]:
from IPython.display import display, HTML
display(HTML(variables.animation_detector_html))